In [1]:
import cv2
import numpy as np
import sys
import matplotlib
from matplotlib import pyplot as plt
import pylab as pl

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [2]:
def normalize_grayscale(image, beta=0.75, image_id_k = 1, A_kp=None):
    N,M = image.shape
    k = image_id_k
    
    if image_id_k > 1 and  A_kp is None :
        print("please define A_kp as this is not the first image")
    elif image_id_k == 1:
        A_kp = np.zeros((N,M))
        
    alpha = beta*(k-1)/k
    a = np.mean(image, axis=0)
    A_k = alpha*A_kp + (1-alpha)*np.tile(a,(N,1))
    image = 128*(image/A_k)
    return image, A_k

In [5]:
mytrainX = []
mytrainY = np.concatenate((np.ones(1000), np.zeros(1000)),0)
A_kp=None
for i in range(1,1001):
    txt = '/Users/delinwang/Desktop/Concrete Crack Images for Classification.rar Folder/Positive/0'+str(i).zfill(4)+'.jpg'
    img = cv2.imread(txt, 0).astype(np.float32)
    img2, A_kp = normalize_grayscale(img, beta=0.75, image_id_k = i, A_kp=A_kp)
    img2 = np.uint8(img2)
    blur = cv2.GaussianBlur(img2,(5,5),0)
    ret3,myfiltered = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    myfiltered = myfiltered.reshape(-1)
    mytrainX.append(myfiltered)
     
A_kp=None    
for i in range(1,1001):
    txt = '/Users/delinwang/Desktop/Concrete Crack Images for Classification.rar Folder/Negative/0'+str(i).zfill(4)+'.jpg'
    img = cv2.imread(txt, 0).astype(np.float32)
    img2, A_kp = normalize_grayscale(img)
    img2 = np.uint8(img2)
    blur = cv2.GaussianBlur(img2,(5,5),0)
    ret3,myfiltered = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    myfiltered = myfiltered.reshape(-1)
    mytrainX.append(myfiltered) 
mytrainX = np.asarray(mytrainX)

In [6]:
from sklearn import svm
clf = svm.SVC()
clf.fit(mytrainX, mytrainY) 
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(mytrainX, mytrainY) 
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(mytrainX, mytrainY)
logistic = linear_model.LogisticRegression(C=1e5)
logistic.fit(mytrainX, mytrainY)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [7]:
svmpospred = []
logipospred = []
regrpospred = []
knnpospred = []
A_kp=None
for i in range(1,121):
    j  = i + 1600
    txt = '/Users/delinwang/Desktop/Concrete Crack Images for Classification.rar Folder/Positive/0'+str(j).zfill(4)+'.jpg'
    img = cv2.imread(txt, 0).astype(np.float32)
    img2, A_kp = normalize_grayscale(img, beta=0.75, image_id_k = i, A_kp=A_kp)
    img2 = np.uint8(img2)
    blur = cv2.GaussianBlur(img2,(5,5),0)
    ret3,myfiltered = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    myfiltered = myfiltered.reshape(-1)
    
    logipospred.append(logistic.predict([myfiltered]))
    svmpospred.append(clf.predict([myfiltered]))
    knnpospred.append(neigh.predict([myfiltered]))
    regrpospred.append(regr.predict([myfiltered]))
 
        
    
logipred=[]
svmpred = []
regrpred = []
knnpred = []
A_kp=None
for i in range(1,121):
    j  = i + 1600
    txt = '/Users/delinwang/Desktop/Concrete Crack Images for Classification.rar Folder/Negative/0'+str(j).zfill(4)+'.jpg'
    img = cv2.imread(txt, 0).astype(np.float32)
    img2, A_kp = normalize_grayscale(img, beta=0.75, image_id_k = i, A_kp=A_kp)
    img2 = np.uint8(img2)
    blur = cv2.GaussianBlur(img2,(5,5),0)
    ret3,myfiltered = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    myfiltered = myfiltered.reshape(-1)
    
    logipred.append(logistic.predict([myfiltered]))
    svmpred.append(clf.predict([myfiltered]))
    knnpred.append(neigh.predict([myfiltered]))
    regrpred.append(logistic.predict([myfiltered]))


In [8]:
print ('svm pos, neg')
print (sum(svmpospred)/120)
print ((120-sum(svmpred))/120)

print ('logistic pos, neg')
print (sum(logipospred)/120)
print ((120-sum(logipred))/120)

print ('knn pos, neg')
print (sum(knnpospred)/120)
print ((120-sum(knnpred))/120)


print ('linear reg pos, neg')
poscount = 0
for i in regrpospred:
    if i>0.5:
        poscount +=1
print (poscount/120)
count = 0
for i in regrpred:
    if i>0.5:
        count +=1
print ((120-count)/120)

svm pos, neg
[ 1.]
[ 0.]
logistic pos, neg
[ 0.925]
[ 0.48333333]
knn pos, neg
[ 0.75833333]
[ 0.38333333]
linear reg pos, neg
0.9666666666666667
0.48333333333333334
